## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2026-02-28-05-30-46 +0000,startribune,"Takeaways: Wild stumble in second game back, l...",https://www.startribune.com/takeaways-wild-stu...
1,2026-02-28-05-27-20 +0000,bbc,At least 15 killed after military plane carryi...,https://www.bbc.com/news/articles/c0rjpxxpp49o...
2,2026-02-28-05-06-32 +0000,nyt,Former U.S. Air Force Officer Is Accused of Tr...,https://www.nytimes.com/2026/02/28/us/air-forc...
3,2026-02-28-05-01-05 +0000,nyt,"Carney Visiting India, Australia and Japan to ...",https://www.nytimes.com/2026/02/28/world/canad...
4,2026-02-28-05-00-10 +0000,nyt,New Geothermal Technology Aims to Produce Clea...,https://www.nytimes.com/2026/02/28/business/ge...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2589/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
83,trump,30
86,he,17
118,ai,17
395,warner,16
55,bill,15


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
89,2026-02-27-23-01-00 +0000,wsj,Trump ends government use of Anthropic’s AI mo...,https://www.wsj.com/politics/policy/trump-will...,170
10,2026-02-28-04-07-00 +0000,wsj,Bill Clinton said he had no idea of the crimes...,https://www.wsj.com/politics/house-republicans...,108
201,2026-02-27-18-25-13 +0000,nypost,Trump makes sobering admission about Iran talk...,https://nypost.com/2026/02/27/us-news/trump-no...,107
205,2026-02-27-18-03-16 +0000,nypost,Forcing Bill Clinton to testify will backfire ...,https://nypost.com/2026/02/27/us-news/forcing-...,97
218,2026-02-27-17-06-03 +0000,nypost,Defiant Bill Clinton claims he ‘saw nothing’ a...,https://nypost.com/2026/02/27/us-news/bill-cli...,86


## Select top headlines

In [5]:
with open('exclude_sources.txt') as f:
    excluded_sources = {
        line.strip()
        for line in f
        if line.strip() and not line.strip().startswith('#')
    }

top_count = 10
top_rows = []
working = word_scores.copy()
remaining = latest.copy()

while len(top_rows) < top_count and not remaining.empty:
    scored_remaining = remaining.assign(
        score=remaining['title'].apply(
            lambda t: sum(
                working.get(w.lower(), 0)
                for w in re.findall(r'[A-Za-z]+', t)
                if len(w) > 1
            )
        )
    )
    ranked_loop = scored_remaining.sort_values('score', ascending=False)

    if ranked_loop.empty:
        break

    top_story = ranked_loop.iloc[0]
    remaining = remaining.drop(top_story.name)

    if top_story['source'] in excluded_sources:
        continue

    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for word in words:
        working.pop(word, None)

    top_rows.append(top_story[['score', 'pubdate', 'source', 'title', 'link']])

top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
89,170,2026-02-27-23-01-00 +0000,wsj,Trump ends government use of Anthropic’s AI mo...,https://www.wsj.com/politics/policy/trump-will...
129,85,2026-02-27-21-50-27 +0000,nypost,Warner Bros. Discovery CEO David Zaslav calls ...,https://nypost.com/2026/02/27/business/warner-...
287,73,2026-02-27-12-00-00 +0000,wsj,Hybrid cyberattacks have become more common. “...,https://www.wsj.com/articles/new-york-state-el...
266,57,2026-02-27-13-19-18 +0000,bbc,Pakistan defence minister says country in 'ope...,https://www.bbc.com/news/articles/cvgjd7pv4y4o...
283,50,2026-02-27-12-02-50 +0000,nypost,Huckabee tells US embassy staff in Israel to l...,https://nypost.com/2026/02/27/us-news/mike-huc...
169,49,2026-02-27-19-54-00 +0000,wsj,The Pentagon said it shot down a “seemingly th...,https://www.wsj.com/politics/national-security...
10,47,2026-02-28-04-07-00 +0000,wsj,Bill Clinton said he had no idea of the crimes...,https://www.wsj.com/politics/house-republicans...
331,43,2026-02-27-09-47-19 +0000,bbc,Green Party wins Gorton and Denton by-election...,https://www.bbc.com/news/articles/cr453rvy6kvo...
68,39,2026-02-28-00-03-55 +0000,bbc,A 'delicate' balance for Canada and a 'win-win...,https://www.bbc.com/news/articles/c1l7q871gnro...
171,38,2026-02-27-19-52-17 +0000,startribune,"Anderson: Growing up in south Minneapolis, fis...",https://www.startribune.com/dennis-anderson-la...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
